# Libraries + Torch check

In [1]:
import pandas as pd
import requests
import json
from pathlib import Path
import re
from datetime import date
import time
import os
from fastai.tabular import *
import matplotlib.pyplot as plt
import numpy as np
from sklearn import linear_model
import sklearn.metrics
from sklearn.model_selection import train_test_split
import math

In [2]:
import torch

In [3]:
torch.cuda.device(0)

In [4]:
torch.cuda.get_device_name(0)

'GeForce RTX 2080'

# Data Retrieval

In [5]:
cleaned_item_names = [name[:-10] for name in list(os.walk(r'data/wiki_prices'))[0][2]]

In [6]:
cleaned_item_names[0:5]

['Potato%20seed', 'Tin%20ore', 'Gout%20tuber', 'Bronze%20bolts', 'Gold%20seal']

In [7]:
#pd.Series(cleaned_item_names).to_csv('./data/cleaned_item_names', index=False, header=False)

In [8]:
len(cleaned_item_names)

3147

In [9]:
# cleaned_item_names

In [10]:
def get_item_df(item_name, sleppy_time=1):
    base_url = r"https://oldschool.runescape.wiki/api.php?action=query&prop=revisions&rvprop=content&format=json&titles=Module%3AExchange%2F"
    postfix_url = r"%2FData"
    full_url = f'{base_url}{item_name}{postfix_url}'
    resp = requests.get(full_url)
    jsond = resp.json()
    try:
        regex_text = jsond['query']['pages'][next(iter(jsond['query']['pages'].keys()))]['revisions'][0]['*']
        #get all of the ones that have volume as well as price
        m = re.findall(r'[\d]+:[\d]+:[\d]+', regex_text)
        item_df = pd.DataFrame()
        item_df['dt'] = [int(string.split(':')[0]) for string in m]
        item_df[f'{item_name} price'] = [int(string.split(':')[1]) for string in m]
        item_df[f'{item_name} volume'] = [int(string.split(':')[2]) for string in m]
        item_df['date_obj'] = item_df['dt'].apply(lambda x: date.fromtimestamp(x))
        #take the ones that just have price so we can get the most recent price as well
        n = re.findall(r'[\d]+:[\d]+', regex_text)
        last_time = date.fromtimestamp(int(n[-1].split(':')[0]))
        last_price = int(n[-1].split(':')[1])
        item_df.loc[-1, 'date_obj'] = last_time
        item_df.loc[-1, f'{item_name} price'] = last_price
        item_df = item_df.fillna(method='ffill')
        item_df.set_index('date_obj', inplace=True)
        item_df = item_df.drop('dt', axis=1)
        return item_df
    except:
        pass



In [11]:
def retrieve_dfs(xslice = True):
    if xslice:
        item_dfs = [get_item_df(item_name,1) for item_name in cleaned_item_names[40:45]]
    else:
        item_dfs = [get_item_df(item_name,1) for item_name in cleaned_item_names]
    return item_dfs

In [ ]:
item_dfs = retrieve_dfs(False)

In [ ]:
item_dfs

In [ ]:
def save_dfs(item_dfs):
    for df in item_dfs:
        try:
            df.to_csv(f'./data/wiki_prices/{df.columns[0]}.csv')
        except:
            pass

In [ ]:
save_dfs(item_dfs)

In [ ]:
def load_dfs(drop_blank=True):
    item_dfs = []
    for name in list(os.walk('./data/wiki_prices'))[0][2]:
        df = pd.read_csv(f'./data/wiki_prices/{name}')
        df = df.set_index(df.columns[0])
#         if drop_blank:
#             if df.empty:
#                 item_dfs.append(df)
#         else:
#             item_dfs.append(df)
        item_dfs.append(df)
    return item_dfs

In [ ]:
item_dfs = load_dfs()

In [ ]:
len(item_dfs)

In [ ]:
type(item_dfs)

In [ ]:
cleaned_item_names[0]

In [ ]:
for item_df in item_dfs:
    try:
        col = item_df.columns[0]
    except:
        invalid_dfs += 1 
        print(item_df)

In [ ]:
retrieved_names = [item_df.columns[0][:-6] for item_df in item_dfs]

In [ ]:
len(retrieved_names)

In [ ]:
len(item_dfs)

In [ ]:
df = pd.DataFrame({'name': retrieved_names, 'price_df': item_dfs})

# Feature creation

In [ ]:
#item_df_training = tl_df['price_df'][300]

In [ ]:
days_back = 90

rolling_averages = [3,7,14,21,30,60,90]

days_ahead_to_predict = 4

In [ ]:
def create_predictive_features(price_df):
    item_df_training = price_df
    item_name = item_df_training.columns[0]
    for i in range(days_back):
        item_df_training[f'{item_name} {i} days ago'] = item_df_training[item_name].shift(i)
    for i in range(days_back):
        item_df_training[f'{item_name} {i} delta days ago'] = item_df_training[item_name].shift(i) - item_df_training[item_name]
    for rolling_average in rolling_averages:
        item_df_training[f'{item_name} {rolling_average} day rolling average'] = item_df_training[item_name].rolling(rolling_average).mean()
    for rolling_average in rolling_averages:
        item_df_training[f'{item_name} delta {rolling_average} day rolling average'] = item_df_training[item_name] - item_df_training[f'{item_name} {rolling_average} day rolling average']
    item_df_training['Y'] = item_df_training[item_name].shift(-1*days_ahead_to_predict)
    return item_df_training

        

In [ ]:
#price_df = tl_df['price_df'][300]

In [ ]:
def trim_df(item_df_training):
    trimmed_df = item_df_training.dropna(how='any')
    return trimmed_df
    

# SGD LR

In [ ]:
def create_model(tl_df_row):
    trimmed_df = tl_df_row['trimmed_df']
    try:
        X = trimmed_df.drop(['Y'], axis=1)
        y = trimmed_df['Y']
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, shuffle=False)
        regr = linear_model.LinearRegression()
        regr.fit(X_train, y_train)
        y_pred = regr.predict(X_test)
        mean_squared_error = sklearn.metrics.mean_squared_error(y_test, y_pred)
        r2_score = sklearn.metrics.r2_score(y_test, y_pred)
        return (regr, y_pred, y_test, mean_squared_error, r2_score)
    except:
        return (0, 0, 0, 0, 0)

In [ ]:
#plt.scatter(y_test, y_pred)

# "Backtest"

In [ ]:
starting_cash = 10**7

In [ ]:
slippage = 0
thresh_buy = 0.00
thresh_sell = 0
max_volume_buy = 0.01

In [ ]:
def should_buy(row):
    if row['perc predicted increase'] > thresh_buy:
        return True

def should_sell(row):
    if row['perc predicted increase'] < thresh_sell:
        return True
    
def buy_price(row):
    return row['current_price']

def sell_price(row):
    return row['current_price']

def max_buy(row):
    d_stock = max(
        min(
        (row['starting_cash']//buy_price(row)),
        math.floor(row['volume']*max_volume_buy)
        ),
        0
    )
    d_cash = -1 * d_stock * buy_price(row)
    return (d_stock, d_cash)

def max_sell(row):
    d_stock = -1 * max(
        0,
        min(row['starting_stock'],
        math.floor(row['volume'] * max_volume_buy)
        )
    )
    d_cash = d_stock *-1 * sell_price(row)
    return (d_stock, d_cash)

In [ ]:
def get_ending_cash(record_df):
    last_row = record_df.iloc[-1,:]
    cash = last_row['ending_cash']
    cash += sell_price(last_row) * last_row['ending_stock']
    return cash

In [ ]:
def create_backtest_df(tl_df_row):
    try:
        y_pred = tl_df_row['y_pred']
        y_test = tl_df_row['y_test']
        item_df_training = tl_df_row['item_df_training']
        item_name = tl_df_row['name']

        bt_df = pd.concat([pd.Series(y_pred, index= y_test.index, name='Prediction'), y_test.rename(index='Y')], axis=1)
        bt_df['Y + 1'] = bt_df['Y'].shift(-1)
        bt_df['Y - 1'] = bt_df['Y'].shift(1)


        bt_df = bt_df.fillna(method='ffill').fillna(method='bfill')

        volume_column = f'{item_name} volume'
        volume = item_df_training[volume_column]

        bt_df = bt_df.join(item_df_training[volume_column].rename(index='volume'))
        bt_df = bt_df.join(item_df_training[f'{item_name} price'].rename(index='current_price'))
        bt_df['perc predicted increase'] = (bt_df['Prediction'] - bt_df['current_price'])/bt_df['current_price']

        bt_df['starting_stock'] = 0
        bt_df['starting_cash'] = 0
        bt_df['ending_stock'] = 0
        bt_df['ending_cash'] = 0

        bt_df.loc[bt_df.index[0], 'starting_cash'] = starting_cash
        bt_df.loc[bt_df.index[0], 'ending_cash'] = starting_cash

        bt_df['action'] = ''

        copy_df = bt_df.copy()
        record_df = bt_df.copy()

        first_row = True
        for i, row in copy_df.iterrows():
            current_row = row
            if first_row:
                prev_row = current_row
                first_row = False
                continue
            current_row['starting_cash'] = prev_row['ending_cash']
            current_row['starting_stock'] = prev_row['ending_stock']
            cash = current_row['starting_cash']
            stock = current_row['starting_stock']
            if should_buy(current_row):
                d_stock, d_cash = max_buy(current_row)
                cash += d_cash
                stock += d_stock
                current_row['action'] = f'Buying + {d_stock} stock, - {d_cash} cash'
            elif should_sell(row):
                d_stock, d_cash = max_sell(current_row)
                cash += d_cash
                stock += d_stock
                current_row['action'] = f'Selling + {d_stock} stock, + {d_cash} cash'
            else:
                current_row['action'] = 'hold'
            current_row['ending_cash'] = cash
            current_row['ending_stock'] = stock
            record_df.loc[i, :] = current_row
            prev_row = current_row
        roi = get_ending_cash(record_df)/starting_cash
        last_perc_prediction = bt_df
        return (roi, record_df)
    except:
        return (0, 0)


# Predict for right now

In [ ]:
def make_predictions(tl_df_row):
    try:
        item_name = tl_df_row['name']
        regr = tl_df_row['regr']
        item_df_training = tl_df_row['item_df_training']
        current_price = item_df_training.loc[item_df_training.index[-1], f'{item_name} price']
        pred_price = regr.predict(np.array(item_df_training.drop('Y', axis=1).loc[item_df_training.index[-1],:]).reshape(1, -1))[0]
        perc_pred_increase = (pred_price - current_price)/(current_price)
        last_day = item_df_training.index[-1]
        return (perc_pred_increase, last_day, current_price)
    except:
        return(0, 0, 0)

# Filter and find trades

In [ ]:
r2_score_thresh = .85
roi_thresh = 1.05
perc_pred_increase_thresh = 0.03

In [ ]:
batch_size = 100


In [ ]:
df_list = []

In [ ]:
bought = pd.read_csv('data/bought')


In [ ]:
bought['name'].values

In [ ]:
for i in range(0,df.shape[0],batch_size):
    tl_df = df.iloc[i:i+100,:]
    
    tl_df['item_df_training'] = tl_df['price_df'].apply(create_predictive_features)

    tl_df['trimmed_df'] = tl_df['item_df_training'].apply(trim_df)
    

    tl_df['regr'], tl_df['y_pred'], tl_df['y_test'], tl_df['mean_squared_error'], tl_df['r2_score'] = zip(*tl_df.apply(create_model, axis=1))


    tl_df['roi'], tl_df['record_df'] = zip(*tl_df.apply(create_backtest_df, axis=1))


    tl_df.sort_values(by='roi', ascending = False)

    tl_df['perc_pred_increase'], tl_df['last_day'], tl_df['current_price'] = zip(*tl_df.apply(make_predictions, axis=1))


    filter_tl_df = tl_df.copy()

    filter_tl_df = filter_tl_df.dropna(how='any')

    filter_tl_df =filter_tl_df.loc[filter_tl_df['r2_score'].apply(lambda x: type(x)) != list]

    filter_tl_df = filter_tl_df.loc[filter_tl_df['r2_score'] > r2_score_thresh]

    print(filter_tl_df.shape)

    filter_tl_df = filter_tl_df.loc[filter_tl_df['roi'] > roi_thresh]

    print(filter_tl_df.shape)

    filter_tl_df = filter_tl_df.loc[
            (filter_tl_df['perc_pred_increase'] > perc_pred_increase_thresh) \
        | (filter_tl_df['name'].isin(bought['name']))
                                   ]

    print(filter_tl_df.shape)

    print(filter_tl_df)
    df_list.append(filter_tl_df)

In [ ]:
filter_tl_df

In [ ]:
df_list

In [ ]:
trading_df = pd.concat(df_list, axis=0).sort_values(by='perc_pred_increase', ascending=False)
trading_df.columns

In [ ]:
trading_df['pred_price'] = (trading_df['perc_pred_increase'] + 1) * trading_df['current_price']

In [53]:
#trading_df.loc[:,['name', 'mean_squared_error', 'r2_score', 'roi', 'perc_pred_increase','last_day', 'current_price']]

In [54]:
to_buy = trading_df.loc[~trading_df['name'].isin(bought['name'])].loc[:,['name', 'mean_squared_error', 'r2_score', 'roi', 'perc_pred_increase','last_day', 'current_price', 'pred_price']]
to_buy

,name,mean_squared_error,r2_score,roi,perc_pred_increase,last_day,current_price,pred_price
2616,Frog%20slippers,2.496724e+07,0.963813,1.087076,0.040565,2020-08-08,138698.0,1.443243e+05
2125,Willow%20seed,7.844582e+04,0.883652,1.397343,0.038577,2020-08-08,2521.0,2.618253e+03
2225,Raw%20shark,2.375519e+03,0.939239,1.200669,0.033875,2020-08-08,615.0,6.358330e+02
2172,Senntisten%20teleport%20(tablet),1.691738e+05,0.914799,1.059838,0.032893,2020-08-08,8596.0,8.878751e+03
552,Team%20cape%20i,1.979389e+09,0.941284,1.347946,0.032232,2020-08-08,1714101.0,1.769351e+06
1978,Lobster,5.824373e+01,0.916023,1.590359,0.031354,2020-08-08,145.0,1.495464e+02


In [55]:
to_sell = trading_df.loc[trading_df['name'].isin(bought['name'])].loc[:,['name', 'mean_squared_error', 'r2_score', 'roi', 'perc_pred_increase','last_day', 'current_price', 'pred_price']]
to_sell

,name,mean_squared_error,r2_score,roi,perc_pred_increase,last_day,current_price,pred_price
3020,Mole%20slippers,3.470657e+08,0.973710,2.356502,0.067838,2020-08-08,542451.0,5.792499e+05
1627,Wooden%20shield%20(g),7.651991e+06,0.920674,1.107953,0.061206,2020-08-08,89622.0,9.510739e+04
2469,Bottomless%20compost%20bucket,1.368472e+09,0.938839,1.157845,0.051178,2020-08-08,302357.0,3.178311e+05
162,Black%20wizard%20hat%20(g),1.050022e+08,0.953910,1.279001,0.050095,2020-08-08,282657.0,2.968167e+05
403,Black%20shield%20(h1),2.018511e+05,0.937103,1.094687,-0.017193,2020-08-08,9198.0,9.039862e+03
916,Ahrim's%20robeskirt,1.301657e+10,0.854800,1.190805,-0.018774,2020-08-08,1744165.0,1.711421e+06
1868,Iron%20ore,1.196186e+02,0.929166,3.413442,-0.041223,2020-08-08,83.0,7.957852e+01



# Update with actual trades

In [55]:
bought.loc[11] = "Ahrim's%20robeskirt"

In [ ]:
bought

In [56]:
currently_buying = to_buy['name'][0:3]
currently_buying


2616    Frog%20slippers
2125      Willow%20seed
2225        Raw%20shark
Name: name, dtype: object

In [57]:
bought.merge(currently_buying, how='outer')

,name
0,Black%20shield%20(h1)
1,Avantoe%20seed
2,Iron%20ore
3,Black%20wizard%20hat%20(g)
4,Bottomless%20compost%20bucket
5,Mole%20slippers
6,Wooden%20shield%20(g)
7,Ahrim's%20robeskirt
8,Frog%20slippers
9,Willow%20seed


In [58]:

#bought = bought.merge(currently_buying, how='outer')

In [59]:
#bought = bought.drop([0,2,3])

In [60]:
#bought.loc[-1] = 'Bottomless%20compost%20bucket'

In [61]:
bought

,name
0,Black%20shield%20(h1)
1,Avantoe%20seed
2,Iron%20ore
3,Black%20wizard%20hat%20(g)
4,Bottomless%20compost%20bucket
5,Mole%20slippers
6,Wooden%20shield%20(g)
7,Ahrim's%20robeskirt


In [57]:
#bought.merge(currently_buying)
bought.to_csv('data/bought', index=False)

In [58]:
pd.read_csv('data/bought')

,name
0,Black%20shield%20(h1)
1,Avantoe%20seed
2,Iron%20ore
3,Black%20wizard%20hat%20(g)
4,Bottomless%20compost%20bucket
5,Mole%20slippers
6,Wooden%20shield%20(g)
7,Ahrim's%20robeskirt


In [ ]:
trading_

In [ ]:
trading_df.loc[:,['name', 'mean_squared_error', 'r2_score', 'roi', 'perc_pred_increase','last_day']].to_csv('data/trading_8.2.csv')

# fastai tabular data

In [ ]:
procs = [Normalize]

In [ ]:
valid_idx = range(int(len(df)*.8), len(df)-60)

In [ ]:
dep_var = 'Y'

In [ ]:
path = Path(f'./data/wiki_urls/{item_name}')

In [ ]:
trimmed_df['date_obj'] = trimmed_df.index

In [ ]:
trimmed_df['date_obj'] = trimmed_df['date_obj'].apply(lambda x: pd.to_datetime(x))

In [ ]:
time_trimmed_df = trimmed_df.loc[trimmed_df['date_obj'] > pd.to_datetime('2018-05-01')]

In [ ]:
time_trimmed_df[item_name].plot()

In [ ]:
time_trimmed_df = time_trimmed_df.drop(columns=['date_obj'], axis=1)

In [ ]:
trimmed_df = time_trimmed_df

In [ ]:
df = trimmed_df

In [ ]:
df

In [ ]:
data = TabularDataBunch.from_df(path, df, dep_var, valid_idx=valid_idx, procs=procs, cat_names=[])

In [ ]:
learn = tabular_learner(data, layers=[500,100], emb_szs={'native-country': 10}, metrics=mean_squared_error)

In [ ]:
data.show_batch()

In [ ]:
learn.save('stage-0')

In [ ]:
learn.lr_find(end_lr=1e3)

In [ ]:
learn.recorder.plot()

In [ ]:
learn.fit_one_cycle(5, max_lr=1e-2)

In [ ]:
learn.fit_one_cycle(5, max_lr=1e-2)

In [ ]:
learn.show_results(ds_type=DatasetType.Train)

In [ ]:
learn.save('stage-1')

In [ ]:
learn = learn.load('stage-1')

In [ ]:
learn.lr_find()

In [ ]:
learn.recorder.plot()

In [ ]:
learn.fit_one_cycle(5, 3e-5)

In [ ]:
learn.show_results(ds_type=DatasetType.Train)

In [ ]:
learn.save('stage-2')

In [ ]:
learn.lr_find()

In [ ]:
learn.recorder.plot()

In [ ]:
learn.fit_one_cycle(5, 3e-6)

In [ ]:
learn.save('stage-3')


In [ ]:
learn.show_results(ds_type=DatasetType.Train)

In [ ]:
learn.lr_find()
learn.recorder.plot()

In [ ]:
learn.fit_one_cycle(5, 3e-4)

In [ ]:
learn.save('stage-4')

In [ ]:
learn.lr_find(end_lr=1e2)
learn.recorder.plot()

In [ ]:
learn.show_results(ds_type=DatasetType.Train)

In [ ]:
learn.fit_one_cycle(10, 1e-4)

In [ ]:
learn = learn.load('stage-4')

In [ ]:
learn.show_results(ds_type=DatasetType.Train)

# Plot results

In [ ]:
int(learn.predict(df.iloc[0])[1])

In [ ]:
learn.predict(df.iloc[0])

In [ ]:
plot_x = [float(learn.predict(df.iloc[i])[1]) for i in range(len(df))]

In [ ]:
plot_y = df['Y']

In [ ]:
plot_y

In [ ]:
plot_x

In [ ]:
plot_df = pd.DataFrame(plot_y)

In [ ]:
plot_df['x'] = plot_x

In [ ]:
plot_df.head()

In [ ]:
plot_df.plot()

In [ ]:
plt.plot(plot_x)

In [ ]:
plt.plot(plot_y)

In [ ]:
plt.plot(range(len(plot_x)),np.array([plot_x,plot_y]).T)

In [ ]:
len(plot_x)

In [ ]:
plt.plot(range(len(plot_x[len(plot_x)-60:])), np.array([plot_x[len(plot_x)-60:], plot_y[len(plot_y)-60:]]).T)
plt.legend()

In [ ]:
plt.plot(plot_x[len(plot_x)-60:])

In [ ]:
len(plot_x)

In [ ]:
plt.plot(plot_y[len(plot_y)-60:])

In [ ]:
x_series = pd.Series(plot_x, name='Predict')

In [ ]:
type(x_series)

In [ ]:
type(plot_y)

In [ ]:
y_series = plot_y.reindex(x_series.index)

In [ ]:
y_series

In [ ]:
x_series

In [ ]:
pd.concat([y_series, x_series],  axis=1).iloc[1705-60:].plot()

In [ ]:
learn.drop

In [ ]:
learn.save('stage-final')
learn.export()

# Load in the full data (recent rows will have been dropped because Ys were N/A)

In [ ]:
prediction_df = item_df_training

In [ ]:
prediction_df['constant'] = time_trimmed_df['Y'].mean() *scaling_factor

In [ ]:
prediction_df.columns

In [ ]:
prediction_df = prediction_df.fillna(method='ffill').fillna(method='bfill').fillna(0)

In [ ]:
predict_data = TabularDataBunch.from_df(path, prediction_df, dep_var, valid_idx=valid_idx, procs=procs, cat_names=[])

In [ ]:
predict_learn = tabular_learner(data, layers=[500,100], emb_szs={'native-country': 10}, metrics=mean_squared_error)

In [ ]:
predict_learn =predict_learn.load('stage-final')

In [ ]:
predict_learn.show_results(ds_type=DatasetType.Train)

In [ ]:
predict_learn.predict(prediction_df.iloc[0])

In [ ]:
predict_learn.predict(prediction_df.iloc[100])

In [ ]:
prediction_df

In [ ]:
predictions = [float(predict_learn.predict(prediction_df.iloc[i])[1]) for i in range(len(prediction_df))]


In [ ]:
predictions

In [ ]:
prediction_df.tail()

In [ ]:
prediction_df['predictions'] = predictions

In [ ]:
prediction_df['predictions'].tail(30)

In [ ]:
prediction_df

#### 